<img src="https://raw.githubusercontent.com/brazil-data-cube/code-gallery/master/img/logo-bdc.png" align="right" width="128"/>

# <span style="color:#336699">Web Land Trajectory Service (WLTS) - Comparing LULC data sets</span>
<hr style="border:2px solid #0077b9;">

<br/>

<div style="text-align: center;font-size: 90%;">
    Fabiana Zioti<sup><a href="https://orcid.org/0000-0002-7305-6043"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Karine Reis Ferreira<sup><a href="https://orcid.org/0000-0003-2656-5504"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Gilberto R. Queiroz<sup><a href="https://orcid.org/0000-0001-7534-0219"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Alana K. Neves, Felipe Menino Carlos<sup><a href="https://orcid.org/0000-0002-3334-4315"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Felipe Carvalho de Souza<sup><a href="https://orcid.org/0000-0002-5826-1700"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Lorena Santos<sup><a href="https://orcid.org/0000-0003-2612-5859"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Rolf Simoes<sup><a href="https://orcid.org/0000-0003-0953-4132"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>
    <br/><br/>
    Earth Observation and Geoinformatics Division, National Institute for Space Research (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    Contact: <a href="mailto:brazildatacube@inpe.br">brazildatacube@inpe.br</a>
    <br/><br/>
</div>

<br/>

<div style="text-align: justify;  margin-left: 15%; margin-right: 15%;font-size: 75%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>This Jupyter Notebook is a supplement of the following paper:</b>
    <div style="margin-left: 10px; margin-right: 10px; margin-top:10px">
      <p> Zioti, F.; Ferreira, K.R.; Queiroz, G.R.; Neves, A.K.; Carlos, F.M.; Souza, F.C.; Santos, L.; Simoes, R.E; 2021. A platform for land use and land cover data integration and trajectory analysis. </p>
    </div>
</div>

### Table 5: Agreement analysis between TerraClass and Mapbiomas, both for 2014. 
<hr style="border:2px solid #0077b9;">

Import the required librarys

In [ ]:
from wlts import WLTS
from lccs import LCCS
from loguru import logger
from tinydb import TinyDB
import numpy
import pandas
import geopandas
import json
import fiona

## 1. Configurations

To run the codes presented in this document, we need to make some settings. The first one is the definition of the addresses of the **W**eb **L**and **T**rajectory **S**ervice (WLTS) and **W**eb **L**and **C**lassification **S**ystem **S**ervice (WLCSS) services.

Thus, in this example, we will use these services provided by the [Brazil Data Cube project](http://brazildatacube.org/). The addresses are defined below:

In [ ]:
#
# Web Land Trajectory Service (WLTS) URL
#
wlts_service_url = "https://brazildatacube.dpi.inpe.br/wlts/"

#
# Web Land Classification System Service (WLCSS) URL
#
lccs_service_url = "https://brazildatacube.dpi.inpe.br/lccs/"

To conduct the case study, we will perform trajectory extraction considering two collections:

- `MapBiomas Amazônia v5`;
- `TerraClass Amazônia v2`;

In the code below, these collections defined:

In [ ]:
collections = 'terraclass_amazonia-v2,mapbiomas_amazonia-v5'

Information on LULC for these projects is retrieved for the year 2014. For this, we set the date and start and end of the query:

In [ ]:
start_date = 2014
end_date = 2014

Thus, in this example, we will use these services provided by the [Brazil Data Cube project](http://brazildatacube.org/). The addresses are defined below:

In [ ]:
# Define Services
service = WLTS(wlts_service_url, access_token='change-me')
service_lccs = LCCS(lccs_service_url)

Additionally, since we will be using the BDC services, defining an **access token** will also be necessary. This token is mandatory, and once created, gives access to all BDC services and data products.

> If you do not have an access token, you can create one through [Brazil Data Cube explorer](https://brazildatacube.dpi.inpe.br/portal/), the BDC data portal. In the portal, **create an account**, and in the **user settings**, **create your token**.

## 2.  Sample points

Path with the data file of the area of interest. For the case study, LULC trajectories were collected using a systematically generated grid for the São Felix do Xingu/Pará State region in Brazil. In total, `4472` points were generated. All of them are with a spacing of 1km between each point.

In [ ]:
file_path = '../../data/raw_data/study-area_sao-felix-do-xingu/sao-felix-do-xingu_utm_sqr_pts1km.shp'

Reads the shapefile that contains the locations of interest:

In [ ]:
df = geopandas.read_file(file_path)
df_parts = numpy.array_split(df, 150)

## 3.  Retrieving LULC Harmonize trajectories

Prepare tinydb to save the returned trajectories and use the logger module to to assist in monitoring operations:

In [ ]:
logger.add("../../data/derived_data/python-scripts/download_points_{time}.log")

db = TinyDB('../../data/derived_data/python-scripts/points_wlts.json', ensure_ascii=False)
last_row = db.get(doc_id=len(db))

To compare different LULC data sets, it is necessary to harmonize their distinct classification systems and LULC classes. In order for the LULC trajectory to be returned in a single classification system, it is necessary to define the parameter **target_system** in the function **tj**. The code below searches for each location of interest the trajectory in the classification system **Simplified-legend-TM-1**.

In [ ]:
for idx, part in enumerate(df_parts):
    for idx2, row in part.iterrows():
#         
#       Check if this location has already been consulted.
#       If it is a new location, the next step is performed
# 
        if last_row and idx2 <= last_row['id']:
            continue
# 
#       Shows in which step the data query is located
# 
        logger.info(f'Part({idx}) | row({idx2})')

#     
#       Retrieve the LULC path from the WLTS service    
# 
        _res = service.tj(latitude = row.geometry.y,
                          longitude = row.geometry.x,
                          collections = collections,
                          start_date = start_date,
                          end_date = end_date,
                          target_system='Simplified-legend-TM-1')
# 
#       Save the result using tinydb
# 
        db.insert({'id': idx2, **_res})
